In [1]:
import numpy as np
import pandas as pd
#from tqdm import tqdm
import pickle

with open('../input/pretrained-for-riiid/user_ele_dict.pkl', 'rb') as f:
    user_ele_dict = pickle.load(f)

with open('../input/pretrained-for-riiid/user_num_dict.pkl', 'rb') as f:
    user_num_dict = pickle.load(f)

with open('../input/pretrained-for-riiid/ques_ele_vec.pkl', 'rb') as f:
    ques_ele_vec = pickle.load(f)

with open('../input/pretrained-for-riiid/ques_num_vec.pkl', 'rb') as f:
    ques_num_vec = pickle.load(f)  

In [2]:
question_dtype = {
    'question_id':'int16',
    'tags':'object'
}
questions_data = pd.read_csv('../input/riiid-test-answer-prediction/questions.csv',
                             usecols = question_dtype.keys(), 
                             dtype = question_dtype)

questions_data.tags.fillna('92',inplace=True)

def gen_vec(row):
    row['vec'] = np.zeros(188)
    index_list = row.tags.split()
    for index_ in index_list:
        row.vec[int(index_)] = 1.0
    return row

questions_data = questions_data.apply(gen_vec, axis='columns')

In [3]:
def predict_y(row_data, user_ele, user_num, ques_ele=ques_ele_vec, ques_num=ques_num_vec, q_data=questions_data):
    y = np.ones(len(row_data))*0.25
    i = 0
    for index, row in row_data.iterrows():
        mask = q_data.vec[row.content_id]
        ques_prob_vec = ques_ele/ques_num
        if row.user_id in user_ele.keys():
            borrow_index = (user_num[row.user_id] == 0)
            user_prob_vec = user_ele[row.user_id]/user_num[row.user_id]
            user_prob_vec[borrow_index] = ques_prob_vec[borrow_index]
            y[i] = sum(user_prob_vec * mask)/sum(mask)
        else:
            y[i] = sum(ques_prob_vec * mask)/sum(mask)
        i = i + 1
    return y

In [4]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df[test_df.content_type_id == 0]
    test_df['answered_correctly'] = predict_y(test_df,user_ele_dict, user_num_dict)
    #print('-----------------------------------------')
    #print(test_df[['row_id', 'answered_correctly']])
    #print('-----------------------------------------')
    env.predict(test_df[['row_id', 'answered_correctly']])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: invalid value encountered in true_divide
  if __name__ == '__main__':
